In [11]:
import numpy as np
from itertools import combinations


In [12]:
n_samples = 1000000

In [13]:
def generate_random_array(n):
    """
    Generate a 4 x n x 2 array where:
    - For each of the 4 sets:
      - Column 0: values between 0 and pi
      - Column 1: values between 0 and 2*pi
    """
    col1 = np.random.uniform(0, np.pi, (4, n))
    col2 = np.random.uniform(0, 2 * np.pi, (4, n))
    return np.stack((col1, col2), axis=-1)

array = generate_random_array(n_samples)
print(array)


[[[2.9194577  5.99568608]
  [0.25419009 1.08123607]
  [1.76713938 3.57149096]
  ...
  [2.52773786 2.04124133]
  [1.16702624 5.54651572]
  [1.65394787 5.80297998]]

 [[1.82451997 4.19560079]
  [2.99844925 3.84398332]
  [1.77014902 1.50784701]
  ...
  [3.08601045 6.09145591]
  [1.44653793 2.08911218]
  [0.15402917 4.65114918]]

 [[2.69504742 6.11352495]
  [0.88296889 1.75928888]
  [1.20528014 2.10250726]
  ...
  [0.31215341 4.90581603]
  [2.96675263 1.57475711]
  [0.53225828 3.94688867]]

 [[2.33900233 5.50561157]
  [1.08033994 5.33936919]
  [0.56905824 4.51701561]
  ...
  [0.79700475 6.07438226]
  [1.13840089 4.12896393]
  [1.60645766 0.91358313]]]


In [14]:
def spherical_to_cartesian(spherical_array):
    """
    Convert a 4 x n x 2 array of spherical coordinates (theta, phi) with radius=1
    to a 4 x n x 3 array of Cartesian coordinates (x, y, z).
    """
    theta = spherical_array[..., 0]  # polar angle
    phi = spherical_array[..., 1]    # azimuthal angle
    r = 1.0

    x = r * np.sin(theta) * np.cos(phi)
    y = r * np.sin(theta) * np.sin(phi)
    z = r * np.cos(theta)

    return np.stack((x, y, z), axis=-1)

# Example usage:
cartesian_array = spherical_to_cartesian(array)
print(cartesian_array)


[[[ 2.11270108e-01 -6.24707416e-02 -9.75429315e-01]
  [ 1.18246766e-01  2.21924861e-01  9.67867273e-01]
  [-8.91542951e-01 -4.08770615e-01 -1.95083956e-01]
  ...
  [-2.61101323e-01  5.13447493e-01 -8.17433649e-01]
  [ 6.81146886e-01 -6.17800862e-01  3.92888044e-01]
  [ 8.83835763e-01 -4.60365167e-01 -8.30557518e-02]]

 [[-4.78271743e-01 -8.41575938e-01 -2.51010120e-01]
  [-1.08888601e-01 -9.21615004e-02 -9.89772464e-01]
  [ 6.16618579e-02  9.78253546e-01 -1.98034885e-01]
  ...
  [ 5.45356313e-02 -1.05861186e-02 -9.98455707e-01]
  [-4.91598152e-01  8.61957313e-01  1.23938885e-01]
  [-9.38958961e-03 -1.53133242e-01  9.88160941e-01]]

 [[ 4.25651649e-01 -7.29171870e-02 -9.01944432e-01]
  [-1.44773603e-01  7.58942199e-01  6.34860097e-01]
  [-4.73515494e-01  8.05000530e-01  3.57431425e-01]
  ...
  [ 5.90334011e-02 -3.01381493e-01  9.51674447e-01]
  [-6.88978913e-04  1.73949240e-01 -9.84754379e-01]
  [-3.51632144e-01 -3.65911607e-01  8.61663236e-01]]

 [[ 5.12485051e-01 -5.04527350e-01 -6.94

In [15]:
def compute_centroids(cartesian_array):
    """
    Compute the centroid of each set of 4 coordinates in a 4 x n x 3 array.
    Returns an n x 3 array of centroids.
    """
    return np.mean(cartesian_array, axis=0)

# Example usage:
centroids = compute_centroids(cartesian_array)
print(centroids)


[[ 0.16778377 -0.3703728  -0.70580751]
 [ 0.09553124  0.04359096  0.27099586]
 [-0.35200073  0.21147395  0.20168035]
 ...
 [ 0.1380495   0.01330326 -0.04134067]
 [-0.07783122 -0.08491601 -0.01222012]
 [ 0.28333463 -0.04705389  0.43277866]]


In [16]:
def compute_plane_normals(cartesian_array):
    """
    For each set of 4 points (shape: 4 x n x 3), compute the unit normals of the
    4 planes formed by combinations of 3 points. Returns a 4 x 3 x n array.
    """
    n = cartesian_array.shape[1]
    normals = np.zeros((4, 3, n))

    for i, combo in enumerate(combinations(range(4), 3)):
        A = cartesian_array[combo[0]]  # shape: (n, 3)
        B = cartesian_array[combo[1]]
        C = cartesian_array[combo[2]]

        AB = B - A
        AC = C - A

        normal = np.cross(AB, AC)  # shape: (n, 3)
        norm = np.linalg.norm(normal, axis=1, keepdims=True)
        unit_normal = normal / norm

        normals[i] = unit_normal.T  # transpose to get shape (3, n)

    return normals

# Example usage:
normals = compute_plane_normals(cartesian_array)
print(normals)


[[[-1.81122737e-01  9.22238615e-01  7.01512298e-01 ... -8.65188958e-01
   -7.86303357e-01  3.18259357e-01]
  [ 7.50860583e-01  3.50472384e-01 -4.80981597e-01 ... -4.96249537e-01
   -6.16868688e-01 -8.07327007e-01]
  [ 6.35140094e-01 -1.63233099e-01  5.25868044e-01 ... -7.20032191e-02
    3.46417781e-02  4.96924627e-01]]

 [[ 1.48107371e-01 -8.68668686e-01  6.77736179e-01 ... -7.87348918e-01
    5.47599522e-04 -7.80021927e-01]
  [ 5.99945734e-01 -4.63402615e-01 -4.66969144e-01 ... -5.61880935e-01
    1.79242569e-01 -1.47304573e-01]
  [ 7.86212009e-01  1.75136321e-01 -5.67990748e-01 ...  2.53715386e-01
    9.83804758e-01 -6.08167047e-01]]

 [[ 2.85525652e-01 -9.08843598e-01  7.96774571e-01 ... -4.82824961e-02
   -8.83044906e-02 -6.09152337e-01]
  [-3.67296601e-01 -4.14129923e-01  6.57705414e-04 ...  9.03879899e-01
    8.44120244e-01 -1.03243842e-01]
  [-8.85193939e-01  4.99972077e-02 -6.04276304e-01 ...  4.25052854e-01
    5.28832043e-01 -7.86304101e-01]]

 [[-2.28896255e-01  8.85009476e

In [17]:
def compute_plane_constants(cartesian_array, normals):
    """
    Compute the constants d in the plane equation n·x = d for each of the 4 planes
    per group of 4 points. Returns an n x 4 array.
    """
    n = cartesian_array.shape[1]
    constants = np.zeros((n, 4))

    for i, combo in enumerate(combinations(range(4), 3)):
        A = cartesian_array[combo[0]]  # shape: (n, 3)
        n_vec = normals[i].T           # shape: (n, 3)
        d = np.einsum('ij,ij->i', n_vec, A)  # dot product per row
        constants[:, i] = d

    return constants

# Example usage:
constants = compute_plane_constants(cartesian_array, normals)
print(constants)


[[-0.7047069  -0.77308264  0.94671244  0.71064745]
 [ 0.02884229 -0.03604911 -0.15098288  0.06342101]
 [-0.53140562 -0.30254177 -0.59274299  0.30466807]
 ...
 [ 0.02996176 -0.29031401  0.12924899 -0.16523808]
 [-0.14087574  0.27616191 -0.37387476  0.3553321 ]
 [ 0.61168178 -0.57108561 -0.42555367 -0.2188452 ]]


In [18]:
def compute_centroid_dot_normals(centroids, normals):
    """
    For each group (n groups), compute dot product of the centroid with each of the 4 unit normals.
    Returns an n x 4 array.
    """
    n = centroids.shape[0]
    dot_values = np.zeros((n, 4))

    for i in range(4):
        n_vec = normals[i].T           # shape: (n, 3)
        dot = np.einsum('ij,ij->i', n_vec, centroids)
        dot_values[:, i] = dot

    return dot_values

# Example usage:
dot_centroids = compute_centroid_dot_normals(centroids, normals)
print(dot_centroids)


[[-0.75677445 -0.75226791  0.80871977  0.73749749]
 [ 0.05914453 -0.05572394 -0.09132624  0.04850965]
 [-0.24259067 -0.45186801 -0.4021968   0.02134453]
 ...
 [-0.12306398 -0.12665674 -0.0122128  -0.01725787]
 [ 0.11315765 -0.0272854  -0.07126887  0.08196711]
 [ 0.34322015 -0.47727769 -0.50803157  0.05701478]]


In [19]:
def test_dot_vs_constants(dot_centroids, constants):
    """
    For each value in dot_centroids and constants:
    - If d > 0, return 1 if dot_cent < d, else 0
    - If d < 0, return 1 if dot_cent > d, else 0
    Returns an n x 4 array of 1s and 0s.
    """
    result = np.where(constants > 0,
                      dot_centroids < constants,
                      dot_centroids > constants)
    return result.astype(int)

# Example usage:
test_result = test_dot_vs_constants(dot_centroids, constants)
print(test_result)


[[0 1 1 0]
 [0 0 1 1]
 [1 0 1 1]
 ...
 [1 1 1 1]
 [1 1 1 1]
 [1 1 0 1]]


In [20]:
def reduce_to_single_flag(test_result):
    """
    Reduce an n x 4 array to an n x 1 array:
    - 1 if all 4 entries in the row are 1
    - 0 otherwise
    """
    return np.all(test_result == 1, axis=1).astype(int).reshape(-1, 1)

# Example usage:
final_result = reduce_to_single_flag(test_result)
print(final_result)


[[0]
 [0]
 [0]
 ...
 [1]
 [1]
 [0]]


In [21]:
def compute_average_flag_value(final_result):
    """
    Compute the average value of the final_result array (n x 1).
    """
    return np.mean(final_result)

# Example usage:
average_value = compute_average_flag_value(final_result)
print(average_value)


0.125034
